In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
from seismic_purifier.config import BATCH_SIZE
from seismic_purifier.representation_learning_models import (
    RepresentationLearningSingleAutoencoder,
    RepresentationLearningDenoisingSingleAutoencoder,
    RepresentationLearningMultipleAutoencoder
)
from seismic_purifier.classifier_models import (
    ClassifierAutocovariance, 
    ClassifierAugmentedAutoencoder, 
    ClassifierMultipleAutoencoder
)
import matplotlib.pyplot as plt

2025-04-28 13:56:56.221565: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-28 13:56:56.266686: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-28 13:56:56.266717: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-28 13:56:56.266745: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-28 13:56:56.273759: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-28 13:56:56.274427: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, roc_auc_score

In [3]:
tf.keras.backend.clear_session()
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        # Limit memory growth to avoid consuming all GPU memory
        for device in physical_devices:
            tf.config.experimental.set_memory_growth(device, True)
    except RuntimeError as e:
        print(e)

In [4]:
# ============================
# 1. Configuration
# ============================

# Paths to your data
TEST_DATA_PATH = "/home/ege/Documents/STEAD_SUBSAMPLER/SLVR_NPY/KO.SLVT.2019-09-24T00:00:00.000000Z-2019-09-29T00:00:00.080000Z.npy"#'/home/ege/Documents/STEAD_SUBSAMPLER/test_data_1000_n90e10.npy'  # Replace with your actual path
TEST_LABEL_PATH = '/home/ege/Documents/STEAD_SUBSAMPLER/test_labels_1000_n90e10.npy'  # Replace with your actual path

# Paths to your model
MODEL_PATH = '/home/ege/rawcovar/npy_analysis/checkpoints_SLVT_5epocs_full/SLVT_FULL_representation_cross_covariances_5epochs.h5'

In [5]:
# ============================
# 2. Data loading
# ============================
X_test = np.load(TEST_DATA_PATH)
print(f"Test data shape: {X_test.shape}")

Y_test = np.load(TEST_LABEL_PATH)  # Expected shape: (num_samples)
print(f"Test label shape: {Y_test.shape}")

Test data shape: (14400, 3000, 3)
Test label shape: (1000,)


In [6]:
# ============================
# 3. Representation Learning Model Instantiation
# ============================

# Choose the model you want to train
# For example, using RepresentationLearningSingleAutoencoder
#model = RepresentationLearningSingleAutoencoder(
# name="rep_learning_autoencoder"
#)

# Alternatively, you can choose other models:
# model = RepresentationLearningDenoisingSingleAutoencoder(
#     name="rep_learning_denoising_autoencoder",
#     input_noise_std=1e-6,
#     denoising_noise_std=2e-1
# )
model = RepresentationLearningMultipleAutoencoder(
     name="rep_learning_autoencoder_ensemble",
     input_noise_std=1e-6,
     eps=1e-27
)

In [7]:
# ============================
# 4. Model Compilation
# ============================
model.compile()
model(X_test)

2025-04-14 16:15:02.222206: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1036800000 exceeds 10% of free system memory.
2025-04-14 16:15:03.089691: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 518400000 exceeds 10% of free system memory.
2025-04-14 16:15:03.269148: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 518400000 exceeds 10% of free system memory.
2025-04-14 16:15:03.562064: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 518400000 exceeds 10% of free system memory.
2025-04-14 16:15:03.916513: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 518400000 exceeds 10% of free system memory.


(<tf.Tensor: shape=(14400, 94, 64), dtype=float32, numpy=
 array([[[ 0.00221819,  0.18285751,  0.5087169 , ..., -0.14084277,
           0.7720012 , -0.59785116],
         [ 0.11278833,  0.18906672,  0.2351128 , ..., -0.14312117,
           0.73331183, -0.5894014 ],
         [ 0.00553132,  0.07162559,  0.45172328, ..., -0.23794799,
           0.7329268 , -0.47248146],
         ...,
         [ 0.01273589,  0.08393856,  0.14060147, ..., -0.01958111,
           0.19729185, -0.12474623],
         [ 0.0217523 ,  0.07931181,  0.15807685, ...,  0.0696976 ,
           0.14242326, -0.14759897],
         [-0.054229  ,  0.06777272,  0.05912469, ..., -0.01131967,
           0.22725393, -0.162631  ]],
 
        [[ 0.00907076,  0.11815607,  0.1051099 , ...,  0.00188299,
           0.2547431 , -0.2878587 ],
         [ 0.0772974 ,  0.11190473,  0.09245374, ...,  0.0061143 ,
           0.29991046, -0.21668832],
         [ 0.02729621,  0.1631362 ,  0.1408867 , ...,  0.00553034,
           0.30021274, -0.

In [8]:
print(np.any(np.isnan(X_test)))
np.isnan(X_test).sum()


False


0

In [9]:
print(np.any(np.isnan(Y_test)))
np.isnan(Y_test).sum()

False


0

In [10]:
model.load_weights(MODEL_PATH)

In [11]:
# ============================
# 5. Classifier Model Instantiation
# ============================

# Choose the model for classification. This is just for convenience, these models are actually wrappers around
#representation learning models.
# For example, using RepresentationLearningSingleAutoencoder
# model_classifier = ClassifierAutocovariance(model)

# Alternatively, you can choose other wrappers. 
# model_classifier = ClassifierAugmentedAutoencoder(model)
model_classifier = ClassifierMultipleAutoencoder(model)

"""Note: One should be careful about the compatibility of the classifier wrappers with the models. RepresentationLearningSingleAutoencoder and RepresentationLearningDenoising
Autoencoder are compatible with ClassifierAutocovariance, ClassifierAugmentedAutoencoder. However, RepresentationLearningMultipleAutoencoder is only compatible with 
ClassifierMultipleAutoencoder. """

'Note: One should be careful about the compatibility of the classifier wrappers with the models. RepresentationLearningSingleAutoencoder and RepresentationLearningDenoising\nAutoencoder are compatible with ClassifierAutocovariance, ClassifierAugmentedAutoencoder. However, RepresentationLearningMultipleAutoencoder is only compatible with \nClassifierMultipleAutoencoder. '

In [12]:
# ============================
# 6. Obtain earthquake probabilities
# ============================
earthquake_scores = model_classifier(X_test)

In [14]:
earthquake_scores

array([0.08577311, 0.04892594, 0.04029876, ..., 0.05423534, 0.07079273,
       0.06726438], dtype=float32)

In [ ]:
# ============================
# 7. Plot ROC curve.
# ============================
fpr, tpr, __ = roc_curve(Y_test, earthquake_scores)
auc_score = roc_auc_score(y_true=Y_test, y_score=earthquake_scores)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC Curve (AUC = {auc_score:.3f})')
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--', label='Random Classifier')

plt.xlabel('False Positive Rate (FPR)', fontsize=14)
plt.ylabel('True Positive Rate (TPR)', fontsize=14)
plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=16)
plt.legend(loc='lower right', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()